[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/drive/run.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get -y install -qq aria2
!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
!pip install -q --pre triton

%cd /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui

!git reset --hard
!git pull

!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/launch.py

!python launch.py --share --xformers --enable-insecure-extension-access --gradio-queue --ui-config-file /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/ui-config.json --ui-settings-file /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/config.json